In [ ]:
import os.path
import subprocess

import pandas as pd

from kcmc_instance import KCMC_Instance

from gurobi_models import gurobi_multi_flow, gurobi_single_flow, get_installation


from tqdm.notebook import tqdm
    


list_serialized_instances = pd.read_csv('/data/instances.csv', header=None)[0].tolist()[:1000]

K = 3
M = 2

GUROBI_TIMEOUT = 60
GUROBI_THREADS = 4

In [ ]:
def get_cache(prefix):
    file = f'/data/{prefix}_{K}_{M}.pq'
    if os.path.exists(file):
        data = pd.read_parquet(file)
        cache = set(data['serialized_instance'].unique())
        return data.to_dict(orient='records'), cache
    return [], set()

def to_cache(prefix, df):
    file = f'/data/{prefix}_{K}_{M}.pq'
    df.to_parquet(file)

In [ ]:
gurobi_mf_data, cache = get_cache('gurobi_mf')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue
    try:
        obj_instance = KCMC_Instance(ser_instance, False, True, True)
    except: continue
        
    model, X, Y = gurobi_multi_flow(K, M, obj_instance, GUROBI_TIMEOUT, GUROBI_THREADS)
    results = model.optimize()
    
    gurobi_mf_data.append({
        'serialized_instance': ser_instance,
        'gurobi_mf_status': results['status'],
        'gurobi_mf_feasible': results['status'] == 'OPTIMAL',
        'gurobi_mf_runtime': results['gurobi_runtime']
    })
    
    #print('\n\n', obj_instance)
    #print(results['status'], results['gurobi_runtime'])
    if results['status'] in {'OPTIMAL'}:
        wsn, installation = get_installation(X)
        #print(len(wsn), len(installation), round(len(installation)/len(wsn), 3))        
        gurobi_mf_data[-1].update({'gurobi_mf_installation': installation,
                                   'gurobi_mf_size': len(installation),
                                   'gurobi_mf_quality': len(installation)/len(wsn)})
        
gurobi_mf_data = pd.DataFrame(gurobi_mf_data)
to_cache('gurobi_mf', gurobi_mf_data)
len(gurobi_mf_data)

In [ ]:
gurobi_sf_data, cache = get_cache('gurobi_sf')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue
    try:
        obj_instance = KCMC_Instance(ser_instance, False, True, True)
    except: continue
        
    model, X, Y = gurobi_single_flow(K, M, obj_instance, GUROBI_TIMEOUT, GUROBI_THREADS)
    results = model.optimize()
    
    gurobi_sf_data.append({
        'serialized_instance': ser_instance,
        'gurobi_sf_status': results['status'],
        'gurobi_sf_feasible': results['status'] == 'OPTIMAL',
        'gurobi_sf_runtime': results['gurobi_runtime']
    })
    
    #print('\n\n', obj_instance)
    #print(results['status'], results['gurobi_runtime'])
    if results['status'] in {'OPTIMAL'}:
        wsn, installation = get_installation(X)
        #print(len(wsn), len(installation), round(len(installation)/len(wsn), 3))        
        gurobi_sf_data[-1].update({'gurobi_sf_installation': installation,
                                   'gurobi_sf_size': len(installation),
                                   'gurobi_sf_quality': len(installation)/len(wsn)})
        
gurobi_sf_data = pd.DataFrame(gurobi_sf_data)
to_cache('gurobi_sf', gurobi_sf_data)
len(gurobi_sf_data)

In [ ]:
eval_data, cache = get_cache('eval')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue   
    result_evaluator = subprocess.run([
        "time", "-f", "%P %S %U",
        "/builds/instance_evaluator", 
        str(K), str(M), ser_instance
    ], capture_output=True)
    
    assert int(result_evaluator.returncode) == 0
    
    output = result_evaluator.stdout.decode()
    i, j = output.replace('\t', '').split('|')
    status = (i.split(':')[-1].strip(), j.split(':')[-1].strip())
    feasible = status == ('SUCCESS', 'SUCCESS')
    
    cpu, systime, usertime = map(
        float,
        result_evaluator.stderr.decode().replace('%', 'e-2').split(' ')
    )
    
    eval_data.append({
        'serialized_instance': ser_instance,
        'eval_status': status, 'eval_feasible': feasible,
        'eval_runtime': (systime+usertime)*cpu,
        'eval_cpu': cpu, 'eval_sys_time': systime, 'eval_user_time': usertime
    })
    
eval_data = pd.DataFrame(eval_data)
to_cache('eval', eval_data)
len(eval_data)

In [ ]:
df = (eval_data
      .merge(gurobi_mf_data)
      .merge(gurobi_sf_data)
).sort_values('serialized_instance').reset_index(drop=True).copy()

In [ ]:
df_diff = df[
    (df['gurobi_mf_feasible'] != df['eval_feasible'])
  | (df['gurobi_sf_feasible'] != df['eval_feasible'])
  | (df['gurobi_mf_feasible'] != df['gurobi_sf_feasible'])
  | (df['gurobi_mf_size'].fillna(-1) != df['gurobi_sf_size'].fillna(-1))
]

assert len(df_diff) == 0

In [ ]:
VIEW = [#'serialized_instance',
    'gurobi_mf_feasible', 'gurobi_sf_feasible', 'eval_feasible',
    # 'gurobi_mf_status', 'gurobi_sf_status', 'eval_status',
    'gurobi_mf_size', 'gurobi_sf_size',
    'gurobi_mf_runtime', 'gurobi_sf_runtime', 'eval_runtime',
    # 'eval_sys_time', 'eval_user_time',  'eval_cpu'
]



df_diff[VIEW]

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['gurobi_mf_runtime'],
                         mode='lines', name='GUROBY MULTI'))
fig.add_trace(go.Scatter(x=df.index, y=df['gurobi_sf_runtime'],
                         mode='lines', name='GUROBY SINGLE'))
fig.add_trace(go.Scatter(x=df.index, y=df['eval_runtime']*100,
                         mode='lines', name='EVAL x100'))

fig.show()

In [ ]:
df[VIEW].sort_values('gurobi_sf_runtime')

In [ ]:
ROW = df.loc[55]
KCMC_Instance(ROW['serialized_instance'], False, True, True).plot(
    labels=True, installation=ROW['gurobi_mf_installation'], minimal=True)

In [ ]:
KCMC_Instance(ROW['serialized_instance'], False, True, True).plot(
    labels=True, installation=ROW['gurobi_sf_installation'], minimal=True)

In [ ]:
ROW['gurobi_sf_installation']